<a href="https://colab.research.google.com/github/flrain2/Study/blob/main/1_pubmed_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pubmed 가져오기

1. 최대 요청 속도 (Rate Limit)
- IP당 3초에 1회 요청 (1 request per 3 seconds)
- email 파라미터를 포함한 경우 10 requests per second 가능
- API 키 (api_key 사용) 포함 시 최대 10 requests per second 가능

2. 일일 요청 한도
- API 키 없이: IP당 하루 3,000 requests
- API 키 사용 시: IP당 하루 10,000 requests

In [ ]:
# ✅ Step 1: 필요한 라이브러리 설치 (Colab 환경)
!pip install requests tqdm biopython beautifulsoup4

In [ ]:
#!pip install openai

In [ ]:
# ✅ Step 2: 라이브러리 임포트
import os
import time
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from Bio import Entrez
from tqdm import tqdm
from google.colab import drive
from math import ceil
#from googletrans import Translator
import openai

In [ ]:
# ✅ Step 3: Google Drive 마운트
drive.mount('/content/drive')

# ✅ PDF 저장 경로 설정 (Google Drive에 저장)
SAVE_PATH = "/content/drive/MyDrive/Colab Notebooks/pubmed/"
os.makedirs(SAVE_PATH, exist_ok=True)


Mounted at /content/drive


## 수정1. NCBI Entrez API 설정

In [ ]:
# ✅ NCBI Entrez API 설정 (이메일 필수)
Entrez.email = "ooo@gmail.com"  # 본인의 이메일로 변경
Entrez.api_key = "ooo"  # PubMed API 키 (선택 사항)

In [ ]:
# ✅ 전체 논문 수(검색 결과 개수) 가져오기 함수
def get_total_article_count(keyword, year_filter=None):
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept-Language": "en-US,en;q=0.9"
    }
    # 기본 URL 구성: 키워드를 포함하고, year_filter가 주어지면 filter 파라미터도 추가합니다.
    search_url = f"https://pubmed.ncbi.nlm.nih.gov/?term={keyword.replace(' ', '+')}"
    if year_filter:
        search_url += f"&filter={year_filter}"

    response = requests.get(search_url, headers=headers)
    if response.status_code != 200:
        print("PubMed 페이지를 불러오는데 실패했습니다.")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    # PubMed 검색결과 페이지의 전체 결과 수는 보통 "results-amount" 클래스를 가진 div에 있습니다.
    count_div = soup.find("div", class_="results-amount")
    if count_div:
        count_text = count_div.get_text(strip=True)
        # 예: "1,234 results" 와 같이 표시되므로, 숫자만 추출합니다.
        match = re.search(r'([\d,]+)', count_text)
        if match:
            total = int(match.group(1).replace(',', ''))
            return total

    print("전체 논문 수를 추출하지 못했습니다.")
    return None

# ✅ Full Text URL 가져오기 함수 정의 (기존 코드)
def get_full_text_url(pmid):
    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/90.0.4430.93 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate"
    })

    response = session.get(f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/")
    if response.status_code != 200:
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    title_tag = soup.find('h3', class_='title', string=lambda text: text and "Full text links" in text)
    if title_tag:
        links_div = title_tag.find_next('div', class_='full-text-links-list')
        if links_div:
            a_tag = links_div.find('a', href=True)
            if a_tag:
                full_text_url = a_tag.get('href')
                return full_text_url if full_text_url.startswith("http") else urljoin(response.url, full_text_url)
    return None

def search_pubmed_web(keyword, max_results=10):
    articles = []
    per_page = 10  # 한 페이지당 결과 수 (PubMed 기본)
    total_pages = ceil(max_results / per_page)
    base_url = "https://pubmed.ncbi.nlm.nih.gov/"
    headers = {"User-Agent": "Mozilla/5.0"}

    pmid_list = []  # PMID 저장 리스트
    article_details = {}  # 논문 상세 정보 저장
    counter = 1  # 논문 번호 카운터 (1부터 시작)

    for page in range(1, total_pages + 1):
        search_url = f"{base_url}?term={keyword.replace(' ', '+')}&size={per_page}&page={page}"
        response = requests.get(search_url, headers=headers)
        time.sleep(3)  # 요청 간 딜레이 적용
        soup = BeautifulSoup(response.text, "html.parser")

        # 각 논문 정보 가져오기
        articles_on_page = soup.find_all("article", class_="full-docsum")
        for article in soup.find_all("article", class_="full-docsum"):
            pmid_tag = article.find("a", class_="docsum-title")
            if pmid_tag:
                pmid = pmid_tag["href"].split("/")[-2]
                pmid_list.append(pmid)

                title = pmid_tag.text.strip() if pmid_tag else "No title"
                print(f"{counter}: {title}")  # 논문 번호 + 제목 출력
                counter += 1  # 카운터 증가

                # 무료(Free) 논문 여부 확인
                free_span = article.find("span", class_="free-resources spaced-citation-item citation-part")
                free_status = ""
                if free_span and "Free" in free_span.get_text():
                    free_status = "Free"

                # 논문 기본 정보 저장 (무료 여부 포함)
                article_details[pmid] = {
                    "PMID": pmid,
                    "Title": pmid_tag.text.strip() if pmid_tag else "No title",
                    "Journal": article.find("span", class_="docsum-journal-citation full-journal-citation").text.strip()
                               if article.find("span", class_="docsum-journal-citation full-journal-citation") else "No journal",
                    "Author": article.find("span", class_="docsum-authors full-authors").text.strip()
                              if article.find("span", class_="docsum-authors full-authors") else "No author",
                    "Free": free_status
                }

            if len(pmid_list) >= max_results:
                break

        if len(pmid_list) >= max_results:
            break

    # Entrez API를 통해 초록(Abstract)과 언어(Language) 가져오기
    abstracts = {}
    languages = {}

    try:
        handle = Entrez.efetch(db="pubmed", id=",".join(pmid_list), rettype="xml", retmode="text")
        record = handle.read()
        handle.close()
        time.sleep(1)

        soup_xml = BeautifulSoup(record, "xml")
        for article in soup_xml.find_all("PubmedArticle"):
            pmid_tag = article.find("PMID")
            if pmid_tag:
                pmid = pmid_tag.text.strip()

                language_tag = article.find("Language")
                languages[pmid] = language_tag.text.strip() if language_tag else "Unknown"

                abstract_texts = article.find_all("AbstractText")
                abstracts[pmid] = " ".join([abstract.text for abstract in abstract_texts]) if abstract_texts else "No abstract available"

    except Exception as e:
        for pmid in pmid_list:
            abstracts[pmid] = "No abstract available"
            languages[pmid] = "Unknown"

    # 최종 데이터 정리
    for pmid in pmid_list:
        record = article_details.get(pmid, {})
        record["Language"] = languages.get(pmid, "Unknown")
        record["Full_Text_URL"] = get_full_text_url(pmid)
        record["Abstract"] = abstracts.get(pmid, "No abstract available")
        articles.append(record)

    print("finish")
    return articles


## 수정2. 키워드 & 년수

> keyword
 - 원하는 키워드 입력
 - OR AND 입력시: (키워드) OR (키워드)/ (키워드) AND (키워드)

> year
 - 연수 설정 안할때: year = None
 - 연수 1년 설정시: year = "datesearch.y_1"
 - 연수 5년 설정시: year = "datesearch.y_5"



In [ ]:
# 수정 필요
keyword = "MBCT"
year = None
#year = "datesearch.y_5"  # year filter를 문자열로 전달합니다.

# PubMed 검색 (기본)

## 총 검색결과 확인

In [ ]:
total_articles = get_total_article_count(keyword, year)
if total_articles is not None:
    max_results = total_articles
    print(f"키워드 '{keyword}' 검색 결과: 총 {total_articles}개의 논문이 있습니다.")
else:
    max_results = 10  # 기본값
    print("전체 논문 수를 확인할 수 없어 기본값 10를 사용합니다.")

키워드 'MBCT' 검색 결과: 총 761개의 논문이 있습니다.


## 수정3. 검색결과 확인 후 max_results

In [ ]:
# PubMed 검색 (기본)
max_results = 25
search_results = search_pubmed_web(keyword, max_results)


1: Mechanisms of action in mindfulness-based cognitive therapy (MBCT) and mindfulness-based stress reduction (MBSR) in people with physical and/or psychological conditions: A systematic review.
2: Mindfulness-based cognitive therapy: theory and practice.
3: Immediate impact of Mindfulness-Based Cognitive Therapy (MBCT) among women with breast cancer: a systematic review and meta-analysis.
4: Effects of Mindfulness-Based Stress Reduction on employees' mental health: A systematic review.
5: Mindfulness-based cognitive therapy for patients with chronic, treatment-resistant depression: A pragmatic randomized controlled trial.
6: Cognitive effects of MBSR/MBCT: A systematic review of neuropsychological outcomes.
7: Cognitive Behavioral Therapy and Mindfulness-Based Cognitive Therapy for Depressive Disorders.
8: Mindfulness-based Cognitive Therapy for Generalised Anxiety Disorder: a Systematic Review and Meta-analysis.
9: A systematic review and meta-analysis of acceptance- and mindfulness-b

In [ ]:
# search_results가 앞서 수집한 논문 정보를 담은 리스트라고 가정합니다.
df = pd.DataFrame(search_results)

## 번역하고 싶을때만

In [ ]:
"""
# OpenAI API 키 설정 (본인의 API 키로 변경)
openai.api_key = "YOUR_OPENAI_API_KEY"

def translate_text(text, target_language='ko'):
    """
    OpenAI ChatCompletion API를 사용하여 입력 텍스트를 지정한 언어로 번역하는 함수
    """
    prompt = f"Translate the following text into {target_language}:\n\n{text}"
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # 또는 gpt-4 모델 사용 가능
            messages=[
                {"role": "system", "content": "You are a helpful translation assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=1024
        )
        translated_text = response.choices[0].message['content'].strip()
    except Exception as e:
        print("Error during translation:", e)
        translated_text = ""
    return translated_text

# CSV 파일 읽어오기 (파일명은 필요에 따라 변경)
df = pd.read_csv('data.csv')

# 'Abstract' 컬럼의 각 텍스트를 번역하여 'Trans' 컬럼에 저장
df['Trans'] = df['Abstract'].apply(lambda x: translate_text(x))

In [ ]:
# 1. Ref ID 컬럼 추가: DataFrame 맨 앞에 1부터 시작하는 번호를 추가합니다.
df.insert(0, "Ref ID", range(1, len(df) + 1))

# 2. Full_Text_URL 컬럼의 None 값을 "No"로 대체합니다.
df['Full_Text_URL'] = df['Full_Text_URL'].apply(lambda x: "No" if x is None else x)

# 3. 무료 논문 여부를 DataFrame의 Free 컬럼에 반영
# (이미 article_details에서 Free 정보가 있으면 해당 값을 사용하고,
#  Full_Text_URL에 추가 검사를 통해서도 확인합니다.)
if "Free" not in df.columns:
    df["Free"] = df["Full_Text_URL"].apply(
        lambda url: "Free" if url != "No" and ("pmc/articles" in url.lower() or "free" in url.lower()) else ""
    )

# 4. 원하는 컬럼 순서로 재정렬: Ref ID, PMID, Title, Language, Journal, Author, Full_Text_URL, Abstract 순으로
desired_order = ["Ref ID", "Title", "PMID", "Language", "Free","Full_Text_URL", "Journal", "Author",  "Abstract"]
df = df[desired_order]

df[5]

NameError: name 'df' is not defined

In [ ]:
# ✅ CSV 파일 저장 (파일명을 키워드 기반으로 설정)
file_name = f"/content/drive/MyDrive/Colab Notebooks/pubmed/{keyword}.csv"
df.to_csv(file_name, index=False, encoding="utf-8-sig")

# Mesh DB 검색

In [ ]:
total_articles = get_total_article_count(keyword, year, db="mesh")

전체 논문 수를 추출하지 못했습니다.
검색 URL: https://www.ncbi.nlm.nih.gov/mesh/?term=sepsis
